In [1]:
using Pkg
Pkg.activate(".")
# for pkg in ["BlackBoxOptim", "Cubature", "Distributions", "Integrals", "Roots", "StatsPlots", "StatProfilerHTML"]
#     Pkg.add(pkg)
# end
# Pkg.instantiate()

using BlackBoxOptim, Cubature, Distributions, Integrals, Random, Roots, StatProfilerHTML

  Activating project at `C:\Users\jbrig\Documents\research\mapinator_2024`
[ Info: Precompiling BlackBoxOptim [a134a8b2-14d6-55f6-9291-3336d3ab0209]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for FillArraysPDMatsExt [8f4ebe69-86d3-5e4a-a026-ecafd7646e97]
│   exception = Required dependency FillArraysSparseArraysExt [44fc9386-74a0-5599-8935-2652d5510edb] failed to load from a cache file.
└ @ Base loading.jl:1818
┌ Error: Error during loading of extension FillArraysPDMatsExt of FillArrays, use `Base.retry_load_extensions()` to retry.
│   exception =
│    1-element ExceptionStack:
│    Declaring __precompile__(false) is not allowed in files that are being precompiled.
│    Stacktrace:
│      [1] _require(pkg::Base.PkgId, env::Nothing)
│        @ Base .\loading.jl:1825
│      [2] _require_prelocked(uuidkey::Base.PkgId, env::Nothing)
│        @ Base .\loading.jl:1660
│      [3] _require_prelocked(uuidkey::Base.PkgId)
│        @ Base .\loading.jl:165

In [2]:
Random.seed!(0) # for reproducibility
#res = (;placements = placement_rates)
#res = (;placements = [496 66 33 3; 455 196 84 12; 725 618 363 43; 84 148 83 63; 126 66 41 29; 359 258 131 34; 447 206 94 18; 394 523 508 143]) # can also load an adjacency matrix directly

res = (; placements = [696   150   25   16;
1321   670  212   85;
 847   829  499  165;
  40    79   96  177;
 725   530  236  125;
 782   438  186  118;
 254   167   88   72;
 838  1003  622  477])

(placements = [696 150 25 16; 1321 670 212 85; … ; 254 167 88 72; 838 1003 622 477],)

Helper functions for the mathematics:

In [3]:
# define p_vec as e.g.
# [v2/v1 v3/v2 v4/v3 α1 α2 α3 α4 mu1 mu2 mu3 mu4 mus sg1 sg2 sg3 sg4 sgs]

function F(x, ρ, normals, K)
    sum_base = 0.0
    for i in 1:K
        sum_base += ρ[i] * cdf(normals[i], x)
    end
    return sum_base
end

function G(Fim1, Fx, αsum)
    return (Fim1 - Fx) / αsum
end

function κ(i, t, v_rel)
    sum_base = 0.0
    for j in t:i-1
        sum_base += -log(v_rel[j])
    end
    return sum_base
end

function f_integrand(integrals, x, p)
    j = 1
    for i in 1:p.K
        p.pdfs[i] = pdf(p.normals[i], x)
    end
    base_G = G(p.Fx_vec[p.s], F(x, p.ρ, p.normals, p.K), sum(p.α[1:p.s]))
    for t in 1:p.s
        base_exp = exp(-(base_G + κ(p.s, t, p.v_rel)))
        for i in 1:p.K
            integrals[j] = base_exp * p.pdfs[i]
            j += 1
        end
    end
    return integrals
end

function get_integrals(Fx_vec, x_vec, ρ, normals, α, v_rel, k, K)
    # Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    # x_vec[s] = x_{s-1}, so the limits of integration are and must be offset by 1 below
    all_integrals = zeros(K, k, k)
    pdfs = zeros(K)
    for s in 1:k
        prob = IntegralProblem(IntegralFunction(f_integrand, zeros(s * K)), x_vec[s+1], x_vec[s], (; s, pdfs, Fx_vec, ρ, normals, α, v_rel, K))
        sol = solve(prob, CubatureJLh(); reltol = 1e-3, abstol = 1e-3)
        integrals_result = sol.u
        j = 1
        for t in 1:s
            for i in 1:K
                all_integrals[i, t, s] = integrals_result[j]
                j += 1
            end
        end
    end
    return all_integrals
end     

function q(i, t, all_integrals, α, k)
    return sum([(α[t]/sum(α[1:s])) * all_integrals[i, t, s] for s in t:k])
end

function Fx(t, α, v_rel)
    return 1 - sum([-log(v_rel[j])*sum(α[1:j]) for j in 1:t])
end

function pi(t, α)
    return α[t] / sum(α[1:t])
end

pi (generic function with 1 method)

In [4]:
function chisquare(p_vec, placements, k, K, M)
    v_rel = p_vec[1:k-1]
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    μ = p_vec[2k:2k+K-1]
    σ = p_vec[2k+K:2k+2K-1]
    n = p_vec[2k+2K:2k+3K-1]
    ρ = n / sum(n)
    
    ## compute the cutoffs x and the CDF values F(x)

    normals = [truncated(Normal(μ[i], σ[i]), 0, 1) for i in 1:K]
    
    Fx_vec = ones(k) # sets F(x0) = 1 by default; Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    x_vec = ones(k+1) # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0 # TODO: if this case occurs, can we speed up q()?
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        # there is no simple closed-form for F^{-1}(x) so this numerically computes x1, x2, x3
        x_vec[t+1] = find_zero(x -> F(x, ρ, normals, K) - Fx_vec[t+1], 0.5) 
    end
    
    objective = 0.0
    
    ## compute the q_i^t values: probability that a department, conditional on being in tier i, 
    ##     successfully hires an applicant from tier t.

    # TODO: div by zero and negative floating point edge cases in mean
    all_integrals = get_integrals(Fx_vec, x_vec, ρ, normals, α, v_rel, k, K)
    for i in 1:K, t in 1:k
        expectation = n[i] * q(i, t, all_integrals, α, k)
        objective += (placements[i, t] - expectation) ^ 2 / expectation
    end

    return objective
end

chisquare (generic function with 1 method)

In [5]:
NUMBER_OF_TYPES = 4
numtotal = 8

k = NUMBER_OF_TYPES
K = numtotal
M = sum(res.placements)

# upper/lower bound on the value ratios, which should all be less than 1
# if any ratio turns out to be 1.0 or close to it at optimality, this could indicate that a lower tier has a higher value than a higher one
upper = [1.0 for _ in 1:k-1] 
lower = [0.0 for _ in 1:k-1]

# upper/lower bound on variables proportionate to alpha
append!(upper, [1.0 for _ in 1:k])
append!(lower, [0.0 for _ in 1:k])

# upper/lower bound on the mu parameter of truncated normal, which is strictly within [0, 1] as the mean is greater than mu in truncated normal
append!(upper, [1.0 for _ in 1:K])
append!(lower, [0.0 for _ in 1:K])

# upper/lower bound on the sigma parameter of truncated normal
append!(upper, [5.0 for _ in 1:K])
append!(lower, [0.0 for _ in 1:K])

# upper/lower bound on n_i
append!(upper, [30000.0 for _ in 1:K])
append!(lower, [sum(res.placements[i, :]) for i in 1:K])

# @profilehtml
sol_res = @profilehtml bboptimize(p -> chisquare(p, res.placements, k, K, M), SearchRange = [(lower[i], upper[i]) for i in eachindex(upper)], MaxFuncEvals = 100000, TraceInterval = 5)
sol = best_candidate(sol_res)

# benchmark: 176.23s
# with tol=1e-3: 149.40s
# vector-valued integrand with no optimization: 175.10s
# with variable caching: 42.05s
# with distribution caching: 9.62s

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
5.00 secs, 47038 evals, 46945 steps, improv/step: 0.164 (last = 0.1636), fitness=32.457203500

Optimization stopped after 99912 steps and 9.64 seconds
Termination reason: Max number of function evaluations (100000) reached
Steps per second = 10365.39
Function evals per second = 10374.62
Improvements/step = Inf
Total function evaluations = 100001


Best candidate found: [0.504671, 0.588593, 0.630479, 0.958227, 0.819738, 0.603763, 0.993787, 0.999999, 0.624356, 0.454036, 0.151485, 0.497917, 0.544378, 0.411851, 0.350537, 0.36039, 0.263416, 0.192694, 0.110565, 0.254874, 0.328165, 0.375082, 0.2343, 1287.33, 3829.01, 4495.77, 822.358, 2931.69, 2647.0, 1066.39, 5803.75]

Fitness: 19.961759291



[ Info: Wrote profiling output to file://C:\Users\jbrig\Documents\research\mapinator_2024\statprof/index.html .


31-element Vector{Float64}:
    0.5046713977216207
    0.5885926085670863
    0.6304788391955914
    0.9582273042434895
    0.8197379358862357
    0.6037630681663738
    0.9937870325054844
    0.9999990582755423
    0.6243558252392795
    0.4540361407083542
    0.15148472505243848
    0.49791731847185705
    0.5443781298332973
    ⋮
    0.2548739733371241
    0.32816545944390874
    0.3750818933601702
    0.2343001120832445
 1287.3258580462652
 3829.012178566839
 4495.770465927948
  822.358161347444
 2931.6857971292006
 2647.002784359162
 1066.392353908819
 5803.74979179166

In [6]:
function print_metrics_chisquare(p_vec, placements, k, K, M)
    v_rel = p_vec[1:k-1]
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    μ = p_vec[2k:2k+K-1]
    σ = p_vec[2k+K:2k+2K-1]
    n = p_vec[2k+2K:2k+3K-1]
    ρ = n / sum(n)

    normals = [truncated(Normal(μ[i], σ[i]), 0, 1) for i in 1:K]
    
    Fx_vec = ones(k)
    x_vec = ones(k+1)
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        x_vec[t+1] = find_zero(x -> F(x, ρ, normals, K) - Fx_vec[t+1], 0.5) 
    end

    objective = 0.0
    normalizer = 0.0
    
    q_it = zeros(K, k)
    all_integrals = get_integrals(Fx_vec, x_vec, ρ, normals, α, v_rel, k, K)
    for i in 1:K, t in 1:k
        prob = q(i, t, all_integrals, α, k)
        q_it[i, t] = prob
        normalizer += ρ[i] * prob
    end

    round_1_failure = zeros(K)
    for i in 1:K
        round_1_failure[i] = (1 - sum([q_it[i, s] for s in 1:k]))
    end
    
    # TODO: div by zero and negative floating point in mean
    exp_placements = zeros(K, k)
    for i in 1:K, t in 1:k 
        expectation = n[i] * q_it[i, t]
        exp_placements[i, t] = expectation
        objective += (placements[i, t] - expectation) ^ 2 / expectation
    end
    
    println("objective value = ", objective)
    println("success sample size (departments) = ", M)
    println("estimated total samples (departments) = ", sum(n))
    println("estimated unmatched departments = ", sum(n) - M)
    println("estimated probability of any success: ", M / sum(n))
    println("estimated probability of no success: ", 1 - (M / sum(n)))
    println()
    println("estimated fraction of departments of each tier:")
    for i in 1:K
        println("ρ_", i, " = ", ρ[i])
    end
    println()
    println("estimated departments of each tier:")
    for i in 1:K
        println("n_", i, " = ", n[i])
        println("  Successful: ", sum(placements[i, :]))
        println("  Unsuccessful: ", n[i] - sum(placements[i, :]))
    end
    println()
    #println("fractions observed among successful departments in data:")
    #display(sum(placements, dims = 2) ./ M)
    #println()

    for i in 1:k
        println("pi_", i, " = ", pi(i, α))
    end
    println()

    offer_targets = zeros(k, k)
    for t in 1:k, j in 1:t
        offer_targets[t, j] = pi(j, α) * prod([1 - pi(i, α) for i in j+1:t])
    end
    println("Tier selection probabilities for making offers:")
    display(offer_targets)
    println()

    println("Hiring probabilities (q_i^t):")
    display(q_it)
    println()

    println("Probabilities of failing (1 - sum_t q_i^t):")
    display(round_1_failure)
    println()

    for i in 1:k+1
        println("x_", i - 1, " = ", x_vec[i])
    end
    println()
    for i in 1:k
        println("F(x_", i - 1, ") = ", Fx_vec[i])
    end
    println()
    for i in 1:k
        println("α_", i, " = ", α[i])
        println("  Est. graduates: ", α[i] * (sum(n) - 1))
        println("  Successful: ", sum(placements[:, i]))
        println("  Unsuccessful: ", (α[i] * (sum(n) - 1)) - sum(placements[:, i]))
    end
    println()
    println("Total estimated graduates: ", sum(α) * (sum(n) - 1))
    println("Total successful graduates: ", M)
    println("Total estimated unsuccessful graduates: ", (sum(α) * (sum(n) - 1)) - M)
    println()
    println("estimated placement rates:")
    display(exp_placements)
    println()
    println("actual placement rates:")
    display(placements)
    println()
    println("difference between estimated and actual placement rates:")
    display(exp_placements - placements)
    println()
    println("chi-square p-value")
    println(1 - cdf(Chisq((size(placements)[1] - 1) * (size(placements)[2] - 1)), objective))
    println()
end

print_metrics_chisquare (generic function with 1 method)

In [7]:
# ensure the correct metrics function is used based on the selected optimizer
print_metrics_chisquare(sol, res.placements, k, K, M)

for i in 1:k-1
    println("v_", i + 1, "/v_", i, " = ", sol[i])
end

v_base = 1
for i in 1:k
    println("v", i, ": ", v_base)
    if i != k
        v_base = sol[i] * v_base
    end
end

println()

for select_type in 1:k
    println("mean for type ", select_type, ": ", mean(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
    println("stddev for type ", select_type, ": ", std(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
    println()
end
for select_type in k+1:K
    println("mean for sink ", select_type - k, ": ", mean(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
    println("stddev for sink ", select_type - k, ": ", std(truncated(Normal(sol[2k-1+select_type], sol[2k-1+select_type+K]), 0, 1)))
    println()
end

objective value = 19.961759291354866
success sample size (departments) = 12568
estimated total samples (departments) = 22883.297391077336
estimated unmatched departments = 10315.297391077336
estimated probability of any success: 0.5492215472801799
estimated probability of no success: 0.4507784527198201

estimated fraction of departments of each tier:
ρ_1 = 0.056256134596590956
ρ_2 = 0.16732781614156048
ρ_3 = 0.1964651505023459
ρ_4 = 0.035937048201283184
ρ_5 = 0.12811465703680994
ρ_6 = 0.1156740105729379
ρ_7 = 0.046601341392548914
ρ_8 = 0.2536238415559228

estimated departments of each tier:
n_1 = 1287.3258580462652
  Successful: 887
  Unsuccessful: 400.32585804626524
n_2 = 3829.012178566839
  Successful: 2288
  Unsuccessful: 1541.0121785668389
n_3 = 4495.770465927948
  Successful: 2340
  Unsuccessful: 2155.770465927948
n_4 = 822.358161347444
  Successful: 392
  Unsuccessful: 430.358161347444
n_5 = 2931.6857971292006
  Successful: 1616
  Unsuccessful: 1315.6857971292006
n_6 = 2647.00278

4×4 Matrix{Float64}:
 1.0       0.0       0.0       0.0
 0.538946  0.461054  0.0       0.0
 0.402324  0.344178  0.253498  0.0
 0.283876  0.242848  0.178865  0.29441


Hiring probabilities (q_i^t):


8×4 Matrix{Float64}:
 0.537152   0.116274   0.0279595  0.0092447
 0.348496   0.170593   0.0582384  0.0217044
 0.182571   0.197982   0.101297   0.0400142
 0.0542771  0.0920054  0.114657   0.215231
 0.247162   0.180124   0.0829225  0.0418868
 0.294986   0.162854   0.0731545  0.0439047
 0.240357   0.156935   0.0856506  0.0673155
 0.146134   0.168114   0.111318   0.0801698


Probabilities of failing (1 - sum_t q_i^t):


...


actual placement rates:


8×4 Matrix{Int64}:
  696   150   25   16
 1321   670  212   85
  847   829  499  165
   40    79   96  177
  725   530  236  125
  782   438  186  118
  254   167   88   72
  838  1003  622  477


difference between estimated and actual placement rates:


8×4 Matrix{Float64}:
  -4.51022    -0.316952   10.9929    -4.09905
  13.3956    -16.7956     10.9954    -1.89369
 -26.2041     61.0807    -43.5933    14.8948
   4.63525    -3.33864    -1.7107    -0.0028022
  -0.399255   -1.9342      7.10259   -2.2011
  -1.1723     -6.92433     7.64028   -1.78411
   2.3153      0.354727    3.33714   -0.215229
  10.1234    -27.3066     24.064    -11.7147


chi-square p-value
0.5236846198081262

v_2/v_1 = 0.5046713977216207
v_3/v_2 = 0.5885926085670863
v_4/v_3 = 0.6304788391955914
v1: 1
v2: 0.5046713977216207
v3: 0.29704585445416626
v4: 0.18728112550412535

mean for type 1: 0.7170076970819046
stddev for type 1: 0.2089058233496464

mean for type 2: 0.5896997674650597
stddev for type 2: 0.21963799663821124

mean for type 3: 0.457475623318206
stddev for type 3: 0.18480471117739194

mean for type 4: 0.17034896564948246
stddev for type 4: 0.09492669442663

mean for sink 1: 0.4984181706145987
stddev for sink 1: 0.22212194821394646

mean for sink 2: 0.5249686385150918
stddev for sink 2: 0.24578744581237905

mean for sink 3: 0.4591845539078311
stddev for sink 3: 0.25420136218092243

mean for sink 4: 0.38120488331134056
stddev for sink 4: 0.20260572667761806

